In [47]:
import pickle
import pandas as pd
import os 
from tqdm import tqdm 
import numpy as np

In [48]:
with open("../linear_regressor.pkl", "rb") as f:
    model = pickle.load(f)

In [49]:
model

LogisticRegression(C=0.03359818286283781, max_iter=50)

In [50]:
df

,file_name,consistency,tokenized_length,body,injected_pattern
0,EUPATH_Module_12.jsonl,Consistent,3228,"[['OBI_0000299','is','ObjectProperty'],['OBI_0...",NaN
1,HPIO_Module_6.jsonl,Consistent,96,"[['Entity','is','Class'],['Continuant','is','C...",NaN
2,AGRO_Module_20.jsonl,Consistent,3639,"[['BFO_0000050','is','ObjectProperty'],['RO_00...",NaN
3,PLANTSO_Module_19.jsonl,Consistent,3422,"[['NCBITaxon_102136','is','Class'],['NCBITaxon...",NaN
4,PEAO_Module_1.jsonl,Consistent,2563,"[['MS_1000008','is','Class'],['MS_1000070','is...",NaN
...,...,...,...,...,...
1253,AIO_2_CN_Module_4.jsonl,Inconsistent,1309,"[['www.w3.org/2000/01/-schema#','is','Annotati...",AIO
1254,AIO_2_FB-CV_Module_4.jsonl,Inconsistent,3666,"[['RO_0002573','is','ObjectProperty'],['RO_000...",AIO
1255,AIO_2_TRAK_Module_7.jsonl,Inconsistent,1956,"[['www.w3.org/2000/01/-schema#','is','Annotati...",AIO
1256,AIO_2_PORO_Module_4.jsonl,Inconsistent,3707,"[['BFO_0000050','is','ObjectProperty'],['BFO_0...",AIO


In [51]:
def create_dataset(name : str, embeddings):
    dataset = pd.read_csv(f"../data/{name}", header = 0)
    x = [embeddings.get(filename.split(".")[0]+".npy", np.full((100,), np.NaN)) for filename in dataset["file_name"].values.tolist()]
    # Convert list of arrays into a DataFrame (each row corresponds to one array)
    data = pd.DataFrame(x, columns=[f"feature_{i}" for i in range(100)])  # Adjust number of features as needed
    # Add the "y" column from the dataset
    data["y"] = dataset["consistency"].map({"Consistent" : 0, "Inconsistent" : 1})
    data["file_name"] = dataset["file_name"]
    data["injected_pattern"] = dataset["injected_pattern"]
    # Filter out rows where any feature vector is NaN
    data = data.dropna(subset=[f"feature_{i}" for i in range(100)])

    return data

In [52]:
embeddings = {}
for file in tqdm(os.listdir("../data/embeddings")):
    if(".npy" in file):
        array = np.load("../data/embeddings/"+file)
        embeddings[file] = array

100%|██████████| 8308/8308 [00:00<00:00, 8935.02it/s]


In [53]:
test_df = create_dataset("test_data.csv", embeddings)

In [54]:
test_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,y,file_name,injected_pattern
0,-0.000894,0.039823,0.122465,0.075527,0.079825,0.241816,0.079789,0.080515,-0.061197,0.168113,...,0.177890,0.092298,-0.329166,0.138470,-0.103829,-0.224753,-0.095939,0,EUPATH_Module_12.jsonl,NaN
1,0.334044,-0.114616,0.283322,-0.230942,0.061172,0.215552,0.062373,-0.127083,-0.113278,0.024522,...,-0.091560,0.264797,-0.033326,-0.097347,0.224906,-0.400421,0.127923,0,HPIO_Module_6.jsonl,NaN
2,0.098573,0.148217,-0.008458,0.157849,-0.108422,0.197813,0.060799,-0.005865,-0.041674,-0.014617,...,0.090724,0.069226,-0.129420,0.055163,-0.052173,-0.146703,0.084761,0,AGRO_Module_20.jsonl,NaN
3,0.243047,0.142764,-0.089206,0.177496,0.201157,0.384701,-0.119564,0.144920,-0.062278,-0.019183,...,0.065188,0.319289,-0.211975,-0.097246,-0.079045,-0.392249,-0.306587,0,PLANTSO_Module_19.jsonl,NaN
4,-0.204441,-0.111145,-0.138710,0.069385,0.116197,0.116396,-0.270896,0.170017,-0.308475,0.076063,...,0.083322,0.205804,-0.074525,0.099939,-0.069444,-0.416566,-0.106416,0,PEAO_Module_1.jsonl,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,0.226689,0.207701,-0.073613,0.105990,-0.111310,0.146603,-0.071560,0.051197,-0.176966,-0.089287,...,-0.101789,-0.091326,-0.184763,0.012342,-0.002060,0.001554,-0.204669,1,AIO_2_CN_Module_4.jsonl,AIO
1254,0.106602,-0.176470,-0.086822,-0.102335,-0.025575,0.165799,-0.079347,-0.042783,-0.296782,-0.327224,...,-0.136090,0.328974,-0.082358,-0.005698,0.197691,-0.070362,-0.185678,1,AIO_2_FB-CV_Module_4.jsonl,AIO
1255,0.071096,0.208378,-0.084164,0.172828,-0.098668,0.124164,0.135546,0.120892,-0.230810,0.220494,...,-0.167316,0.105811,-0.084437,0.033115,0.207373,-0.040859,-0.157056,1,AIO_2_TRAK_Module_7.jsonl,AIO
1256,0.234382,-0.084065,-0.060590,0.032706,-0.062097,0.250413,-0.044561,0.159801,-0.230317,0.065019,...,-0.182328,0.331249,-0.223793,0.147684,-0.018865,-0.142289,-0.109075,1,AIO_2_PORO_Module_4.jsonl,AIO


In [55]:
pred = model.predict(test_df.drop(["y", "file_name", "injected_pattern"], axis=1))

In [56]:
len(pred)

1253

In [60]:
pred = pd.DataFrame(pred)

In [61]:
pred

,0
0,Inconsistent
1,Consistent
2,Inconsistent
3,Inconsistent
4,Inconsistent
...,...
1248,Consistent
1249,Consistent
1250,Consistent
1251,Consistent


In [62]:
test_df["pred"] = pred

In [63]:
test_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,y,file_name,injected_pattern,pred
0,-0.000894,0.039823,0.122465,0.075527,0.079825,0.241816,0.079789,0.080515,-0.061197,0.168113,...,0.092298,-0.329166,0.138470,-0.103829,-0.224753,-0.095939,0,EUPATH_Module_12.jsonl,NaN,Inconsistent
1,0.334044,-0.114616,0.283322,-0.230942,0.061172,0.215552,0.062373,-0.127083,-0.113278,0.024522,...,0.264797,-0.033326,-0.097347,0.224906,-0.400421,0.127923,0,HPIO_Module_6.jsonl,NaN,Consistent
2,0.098573,0.148217,-0.008458,0.157849,-0.108422,0.197813,0.060799,-0.005865,-0.041674,-0.014617,...,0.069226,-0.129420,0.055163,-0.052173,-0.146703,0.084761,0,AGRO_Module_20.jsonl,NaN,Inconsistent
3,0.243047,0.142764,-0.089206,0.177496,0.201157,0.384701,-0.119564,0.144920,-0.062278,-0.019183,...,0.319289,-0.211975,-0.097246,-0.079045,-0.392249,-0.306587,0,PLANTSO_Module_19.jsonl,NaN,Inconsistent
4,-0.204441,-0.111145,-0.138710,0.069385,0.116197,0.116396,-0.270896,0.170017,-0.308475,0.076063,...,0.205804,-0.074525,0.099939,-0.069444,-0.416566,-0.106416,0,PEAO_Module_1.jsonl,NaN,Inconsistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,0.226689,0.207701,-0.073613,0.105990,-0.111310,0.146603,-0.071560,0.051197,-0.176966,-0.089287,...,-0.091326,-0.184763,0.012342,-0.002060,0.001554,-0.204669,1,AIO_2_CN_Module_4.jsonl,AIO,NaN
1254,0.106602,-0.176470,-0.086822,-0.102335,-0.025575,0.165799,-0.079347,-0.042783,-0.296782,-0.327224,...,0.328974,-0.082358,-0.005698,0.197691,-0.070362,-0.185678,1,AIO_2_FB-CV_Module_4.jsonl,AIO,NaN
1255,0.071096,0.208378,-0.084164,0.172828,-0.098668,0.124164,0.135546,0.120892,-0.230810,0.220494,...,0.105811,-0.084437,0.033115,0.207373,-0.040859,-0.157056,1,AIO_2_TRAK_Module_7.jsonl,AIO,NaN
1256,0.234382,-0.084065,-0.060590,0.032706,-0.062097,0.250413,-0.044561,0.159801,-0.230317,0.065019,...,0.331249,-0.223793,0.147684,-0.018865,-0.142289,-0.109075,1,AIO_2_PORO_Module_4.jsonl,AIO,NaN


In [69]:
PREFIXES = ["AIO","EID","OIL","OILWI","OILWPI","UE","UEWI1","UEWI2","UEWPI","UEWIP","SOSINETO","CSC","OOR","OOD"]

In [96]:
def get_misclassified_percentage(pattern : str):
    df_pattern = test_df[test_df["injected_pattern"] == pattern]
    print(f"Pattern {pattern}: DataPoints {len(df_pattern[df_pattern['pred'] == 'Inconsistent'])} /{len(df_pattern)}")


In [97]:
for pattern in PREFIXES:
    get_misclassified_percentage(pattern)

Pattern AIO: DataPoints 41 /81
Pattern EID: DataPoints 129 /221
Pattern OIL: DataPoints 1 /1
Pattern OILWI: DataPoints 0 /1
Pattern OILWPI: DataPoints 0 /0
Pattern UE: DataPoints 61 /82
Pattern UEWI1: DataPoints 6 /10
Pattern UEWI2: DataPoints 4 /10
Pattern UEWPI: DataPoints 0 /1
Pattern UEWIP: DataPoints 2 /3
Pattern SOSINETO: DataPoints 0 /1
Pattern CSC: DataPoints 147 /210
Pattern OOR: DataPoints 2 /5
Pattern OOD: DataPoints 1 /4
